In [8]:
# 測試一次爬新北 台北 花蓮

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json
import pandas as pd

ua = UserAgent()
headers = {"User-Agent":ua.random}

result = []
last_pages = []

Taipei = []
NewTaipei = []
Hualien = []
# 台北市:3 新北市:4 花蓮縣:22
for i in [3,4,22]:
    data = {"q": "2^1^"+str(i)+"^^^^^^^^^^^^^^0^^9^1^1^0"}
    url = "https://www.hbhousing.com.tw/ajax/dataService.aspx?job=search&path=house"
    res = requests.post(url,data=data,headers=headers)
    soup = BeautifulSoup(res.content)
    last_pages.append(json.loads(soup.text)["a"])

for city,p in zip([3,4,22],last_pages):
    Name = []
    Address = []
    prices = []
    Year = []
    Area = []
    Pattern = []
    land = []
    for page in range(1,p):
        data = {"q": "2^1^"+str(city)+"^^^^^^^^^^^^^^0^^9^1^"+str(page)+"^0"}
        url = "https://www.hbhousing.com.tw/ajax/dataService.aspx?job=search&path=house"
        res = requests.post(url,data=data,headers=headers)
        soup = BeautifulSoup(res.content)

        for i in json.loads(soup.text)['data']:
            Name.append(i['n'])
            Address.append(i['x'])
            prices.append(i['np'])
            Year.append(i['k'])
            Area.append(i['a'])
        for i in json.loads(soup.text)['data']:
            if type(i['p']) == list:
                Pattern.append(i['p'][0]+"房"+i['p'][1]+"廳"+i['p'][2]+"衛")
            elif i['p'] == "車位":
                Pattern.append(i['p'])
            else:
                Pattern.append("未知的東西") 
        for i in range(len(Name)):
            if city == 3:
                land.append("台北市")
            elif city == 4:
                land.append("新北市")
            else:
                land.append("花蓮縣")

    if city == 3:    
        for i in range(len(Name)):
            Taipei.append({"Name":Name[i],"Address":Address[i],"Price":prices[i],"Year":Year[i],"Area":Area[i],"Pattern":Pattern[i],"Store":"住商不動產","Land":land[i]})
    elif city == 4:
        for i in range(len(Name)):
            NewTaipei.append({"Name":Name[i],"Address":Address[i],"Price":prices[i],"Year":Year[i],"Area":Area[i],"Pattern":Pattern[i],"Store":"住商不動產","Land":land[i]})
    else:
        for i in range(len(Name)):
            Hualien.append({"Name":Name[i],"Address":Address[i],"Price":prices[i],"Year":Year[i],"Area":Area[i],"Pattern":Pattern[i],"Store":"住商不動產","Land":land[i]})
            
df_Taipei = pd.DataFrame(Taipei)
df_Taipei.to_csv("Taipei.csv",encoding="utf_8_sig")

df_NewTaipei = pd.DataFrame(NewTaipei)
df_NewTaipei.to_csv("NewTaipei.csv",encoding="utf_8_sig")

df_Hualien = pd.DataFrame(Hualien)
df_Hualien.to_csv("Hualien.csv",encoding="utf_8_sig")

In [10]:
print("縣市頁數",last_pages)
print("台北筆數:",len(Taipei))
print("新北筆數:",len(NewTaipei))
print("花蓮筆數",len(Hualien))

縣市頁數 [548, 1084, 95]
台北筆數: 5470
新北筆數: 10830
花蓮筆數 940
